In [1]:
import numpy as np

# Part 1

Need to count number of rotations (given a starting dial point) that end on dial point 0.

### Data Input and Conversion

In [2]:
# Import raw data as numpy array
data = np.loadtxt(
    fname='Input1.txt',
    dtype='O'
)
# array vector
puzzle_input_raw = data[:]

In [3]:
def convert_input(input_array: np.array) -> np.array:
    """Convert puzzle input into an array of safe rotations.."""
    
    # instantiate output array
    output_array = np.zeros_like(a=input_array, dtype=int)
    
    # test input array for correct prefix
    LR_test = [(x[0].startswith('L') | x[0].startswith('R')) for x in input_array]
    assert not False in LR_test, \
    "one or more input rotations do not start with L or R"
    
    # convert input into integer rotations
    # negative for left rotations, positive for right
    for i, j in enumerate(input_array):
        if j.startswith('L'):
            rotation = j.split('L')  # split into list of ['L', '123']
            # add integer rotation to output
            output_array[i] = -int(rotation[1]) # negative integer
        else:
            rotation = j.split('R')
            output_array[i] = int(rotation[1]) # positive integer
            
    # array of ints
    return output_array

In [4]:
rotations_array = convert_input(puzzle_input_raw)

In [5]:
rotations_array

array([ -5, -37, -32, ...,  29,  47, -34])

### Process Rotations

In [6]:
def process_rotations(input_array: np.array, start: int) -> np.array:
    """From a given starting point, process the given rotations.
    
    Output is an array of resultant dial points for each rotation.
    """
    # instantiate array
    output_array = np.zeros_like(a=input_array, dtype=int)
    
    for i, j in enumerate(input_array):
        # populate output with start and cum. sum. of rotations
        # at each step. Modulo 100 to keep within the circular
        # range of dial points
        output_array[i] =  (start + sum(input_array[:i+1])) % 100
    
    return output_array

In [7]:
points_array = process_rotations(rotations_array, 50)

### Number of 0s

In [8]:
len(points_array[points_array == 0])

982

# Part 2

Need to count number of rotations (given a starting dial point) that end on dial point 0, AS WELL AS rotations that go via dial point 0

### Process Rotations (with additional functionality)

In [9]:
def process_rotations_v2(input_array: np.array, start: int) -> tuple():
    """From a given starting point, process the given rotations.
    
    Function calculates rotations that end on dial point 0, as well as rotations
    that pass through dial point 0
    """
    
    # instantiate arrays
    dial_points_output_array = np.zeros_like(a=input_array, dtype=int)
    pass_zero_counts_output_array = np.zeros_like(a=input_array, dtype=int)
    
    # instantiate current dial point
    current_dial_point = start  # given argument
    
    # instantiate counter for 0 ending, and for 0 passing
    zero_end_count = 0
    zero_pass_count = 0
    
    for i, j in enumerate(input_array):
        
        # calculate and record new dial point after rotation.
        # modulo 100 to keep within the circular range of dial points
        new_dial_point = (current_dial_point + j) % 100
        dial_points_output_array[i] =  new_dial_point
        
        # for zero-ending rotations, increment counter
        zero_end_count += (new_dial_point == 0)
        
        # ===== ZERO PASS LOGIC =====
        
        full_rotations = abs(
            (
                # start point + rotation raw amount...
                (current_dial_point + j) \
                # ... minus ending dial point gives number of rotations * 100
                - new_dial_point
            ) / 100  # divide by 100 to get number of rotations
        )
        
        # correct rotation over-count for dial point zero for both start/end
        full_rotations -= (current_dial_point == new_dial_point == 0)
        
        # correct positive rotation over-count for zero ends
        if new_dial_point == 0:
            full_rotations -= 1 if np.sign(j)==1 else 0
        
        # correct negative rotation over-count for zero starts
        if current_dial_point == 0:
            full_rotations -= 1 if np.sign(j)==-1 else 0
        
        # ===== CONTINUE =====
        
        # increment counter for zero-passing rotations
        zero_pass_count += full_rotations
        pass_zero_counts_output_array[i] = full_rotations
        
        # update dial point for next rotation
        current_dial_point = new_dial_point
    
    # summary prints
    print(f"zero-ends:\t{zero_end_count}")
    print(f"zero-passes:\t{int(zero_pass_count)}")
    print(f"password:\t{zero_end_count + int(zero_pass_count)}")

    return dial_points_output_array, pass_zero_counts_output_array

- zero start, zero end -> both wrong (+1)
- zero start, non-zero end -> positive good, negative wrong (+1)
- non-zero start, non-zero end -> good
- non-zero start, zero end -> positive rotation wrong (+1), negative good

### Number of 0s, and number of Rotations passing through 0

In [10]:
process_rotations_v2(rotations_array, 50)

zero-ends:	982
zero-passes:	5124
password:	6106


(array([45,  8, 76, ..., 76, 23, 89]), array([0, 0, 1, ..., 0, 1, 1]))